In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Attention(Q,K,V) = softmax[(Q.K')/root(d_k)].V

### Q : Query - what i am looking for
### K : Key - what others offer
### V : Value - actual information

In [1]:
import numpy as np

In [9]:
#no. of tokens on sequence
sequence_len = 4

#embedding dimension
d_model = 6

#input embeddings
x = np.random.rand(sequence_len, d_model)
print(x)
print(f"\nShape : {x.shape}")

[[0.17255662 0.89046146 0.06302079 0.03716833 0.64381609 0.42415671]
 [0.7584668  0.01282786 0.15122622 0.76686935 0.6354084  0.55138562]
 [0.26860023 0.36041556 0.61510388 0.98132322 0.60854504 0.20268606]
 [0.11323265 0.03067561 0.34282456 0.15616114 0.91740022 0.36762164]]

Shape : (4, 6)


## Create Q,K,V Matrix

In [14]:
#for easy understanding
d_k = d_model

#creating Q,K,V Matrix
W_Q = np.random.rand(d_model,d_k)
W_K = np.random.rand(d_model,d_k)
W_V = np.random.rand(d_model,d_k)

Q = x @ W_Q
K = x @ W_K
V = x @ W_V

print(f"\nQ : \n{Q} \nShape : {Q.shape}")
print(f"\nK : \n{K} \nShape : {K.shape}")
print(f"\nV : \n{V} \nShape : {V.shape}")


Q : 
[[0.59006109 1.82351479 1.15275307 0.90101941 1.4495238  0.89537097]
 [1.18709951 2.17988632 0.82164891 1.07656732 1.07170281 1.78281354]
 [0.84124281 2.00528123 1.08753234 1.60723069 1.01350603 2.10906083]
 [0.72427638 1.38246253 0.75056761 0.42002978 0.88752102 1.10903567]] 
Shape : (4, 6)

K : 
[[1.20633098 1.22980092 0.81588463 1.65863819 0.81204501 1.21145426]
 [1.8933115  1.09048379 1.28882706 1.86432664 0.67407991 1.16468786]
 [1.41679902 1.59446406 1.56402334 2.15958336 0.90076985 1.15225886]
 [1.20676985 0.83123506 0.60793939 1.36312051 0.29812121 1.05755017]] 
Shape : (4, 6)

V : 
[[0.64947696 0.91295107 0.89814051 0.78619879 1.36767371 0.86581651]
 [1.13008024 1.20791083 1.84199164 0.61171709 1.52075456 1.04117954]
 [1.20012526 1.19036717 1.65385169 1.00103916 1.57006017 1.28869889]
 [0.38953295 0.8545175  1.11579888 0.51270912 1.00355198 0.74014679]] 
Shape : (4, 6)


## Calculate Score - Q.K'

In [32]:
print("Shape of Q.K' : ")
print(f"{Q.shape} x {K.shape[::-1]} --> (4,4)")

#calculate score
score = Q @ K.T  #K.T as we take the transpose of matrix K

print("\nScore : \n",score)
print("\nShape : ",score.shape)

Shape of Q.K' : 
(4, 6) x (6, 4) --> (4,4)

Score : 
 [[ 7.65112737  8.29109917  9.82967244  5.5358729 ]
 [ 9.59893547 10.48955096 11.78726702  7.41646968]
 [10.41207232 11.31708706 12.90421514  8.06663468]
 [ 5.94717774  6.51919882  7.38878383  4.48948676]]

Shape :  (4, 4)


## Scale the score - (Q.K')/root(d_k)

In [31]:
scaled_score = score/np.sqrt(d_k)
print("Scaled score : \n",scaled_score)

Scaled score : 
 [[3.12355967 3.38482706 4.01294697 2.26001065]
 [3.918749   4.28234125 4.81213161 3.02776107]
 [4.25071072 4.62018145 5.26812377 3.29318982]
 [2.42792515 2.66145177 3.01645837 1.8328253 ]]


## Apply Softmax - softmax[(Q.K')/root(d_k)]

In [33]:
def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x/np.exp(np.sum(exp_x, axis=1, keepdims=True))

In [35]:
attention_weights = softmax(scaled_score)

print(f"Attention Weights : \n {attention_weights}")
print("Shape : ",attention_weights.shape)

Attention Weights : 
 [[0.0494322  0.06419141 0.12030009 0.02084373]
 [0.04692134 0.06749576 0.11464669 0.01924949]
 [0.04779503 0.06915791 0.13220249 0.0183458 ]
 [0.04280898 0.05406973 0.07711371 0.02360947]]
Shape :  (4, 4)


## Weighted sum - W.V

In [36]:
attention_output = attention_weights @ V
print(f"Attention Output : \n{attention_output}")
print("Shape : ",attention_output.shape)

Attention Output : 
[[0.25714101 0.28367928 0.38485301 0.20924239 0.37502264 0.280092  ]
 [0.25183866 0.27728644 0.37755578 0.20281303 0.36613764 0.27289299]
 [0.27500159 0.30021742 0.40942847 0.22162739 0.39651703 0.29733531]
 [0.19064936 0.21636225 0.29192253 0.15603038 0.285542   0.21021185]]
Shape :  (4, 6)


## Full Function for complete process

In [51]:
def self_attention(x):
    d_model = x.shape[1]
    
    W_Q = np.random.rand(d_model, d_model)
    W_K = np.random.rand(d_model, d_model)
    W_V = np.random.rand(d_model, d_model)
    
    Q = x @ W_Q
    K = x @ W_K
    V = x @ W_V
    
    scores = Q @ K.T
    scaled_scores = scores / np.sqrt(d_model)
    
    attention_weights = softmax(scaled_scores)
    output = attention_weights @ V
    
    return attention_weights,output

In [52]:
seq_len = 5
d_model = 8
x = np.random.rand(seq_len, d_model)

attention_weights, output = self_attention(x)

print(f"\nAttention weights : \n{attention_weights}")
print("Shape : ",attention_weights.shape)

print(f"\nOutput : \n{output}")
print("Shape : ",output.shape)


Attention weights : 
[[0.20394829 0.0002913  0.0002333  0.04195778 0.07782443]
 [0.13833924 0.00257023 0.00230323 0.05121787 0.07921086]
 [0.12268867 0.00331633 0.00348641 0.049309   0.07861334]
 [0.17799862 0.0007156  0.00064987 0.04528137 0.0825765 ]
 [0.1893703  0.00042724 0.00039155 0.04342552 0.08150722]]
Shape :  (5, 5)

Output : 
[[0.58595281 0.97859754 0.89114927 0.53737505 0.87462587 0.79174393
  0.80158687 0.98525222]
 [0.48056743 0.80756989 0.73885692 0.44625451 0.73079305 0.65623331
  0.6595192  0.80147006]
 [0.44738383 0.75489593 0.6913082  0.41816882 0.68455632 0.61445175
  0.61554037 0.74625931]
 [0.5485171  0.92042016 0.83964843 0.50723422 0.826052   0.74602764
  0.75229645 0.9213157 ]
 [0.56539553 0.94742331 0.86359237 0.52146327 0.84854597 0.76734886
  0.77489202 0.95070465]]
Shape :  (5, 8)
